In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os

from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.layouts import row

%matplotlib inline


# PCAs - general data exploration
* PUDs and TUDs
* Area inundation from SLR 

### PUD and TUD distributions for the PCA network

PCA shapefile for PUDs and TUDs is the one provided here: https://abag.ca.gov/priority/conservation/, 

dissolved versions dissolved on 'joinkey', which appeared to be a unique identifier.

non-dissolved versions don't dissolve, and use the concat of ('joinkey', 'fipco') as the unique id, which matches what bcdc did during their slr intersections...

In [ ]:
# pudtable = pd.read_csv('../flickr/userdays_avg_annual_bypid.csv')
pudtable = pd.read_csv('../flickr/not_dissolved/userdays_avg_annual_bypid.csv')
pudtable.rename(columns={'avg_ann_ud':'pud'}, inplace=True)

In [ ]:
# tudtable = pd.read_csv('../twitter/userdays_avg_annual_bypid.csv')
tudtable = pd.read_csv('../twitter/tud_nondissolved_pcas/userdays_avg_annual_bypid.csv')
tudtable.rename(columns={'avg_ann_ud':'tud'}, inplace=True)

In [ ]:
data = pudtable.merge(tudtable, on='pid')

In [ ]:
data.shape

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(14,5))
data.hist('pud', bins=50, ax=axs[0])
data.hist('tud', bins=50, ax=axs[1])

In [ ]:
data['logpud'] = np.log1p(data['pud'])
data['logtud'] = np.log1p(data['tud'])

### PUD and TUD are correlated, but capture slightly different visitation patterns

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(14,5))
data.plot.scatter(x='pud', y='tud', ax=axs[0])
data.plot.scatter(x='logpud', y='logtud', ax=axs[1])

### PUD, TUD are influenced by area of PCA

In [ ]:
# pca_shp = gpd.read_file('../twitter/shp/Priority_Conservation_Areas_current_dissolve_pid.shp')
pca_shp = gpd.read_file('../twitter/tud_nondissolved_pcas/shp/Priority_Conservation_Areas_current_pid.shp')

In [ ]:
pca_shp['area_m2'] = pca_shp.area
data = data.merge(pca_shp[['area_m2', 'pid', 'name', 'joinkey', 'fipco']], on='pid')

In [ ]:
data.hist('area_m2', bins=50)

In [ ]:
data['logarea'] = np.log1p(data['area_m2'])

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(14,5))
data.plot.scatter(x='logarea', y='logpud', ax=axs[0])
data.plot.scatter(x='logarea', y='logtud', ax=axs[1])

In [ ]:
data['pud_per_m2'] = data['pud']/data['area_m2']
data['tud_per_m2'] = data['tud']/data['area_m2']

#### break visitation density measures into deciles, so we can easily glance at the top 10% of PCAs, for example

In [ ]:
# data['pudden_tile'] = pd.qcut(data['pud_per_m2'], q=10, labels=False)
# data['tudden_tile'] = pd.qcut(data['tud_per_m2'], q=10, labels=False)

### Add Designation type of each PCA to the table

#### load some PCA metadata - the 'designation' of each PCA as:
NL = Natural Landscape  
AL = Agricultural Land  
UG = Urban Greening  
RR = Regional Recreation

In [ ]:
designation = pd.read_csv('../pca/FINAL_List_of_165_PCAs_Designations.csv')
# Source: https://abag.ca.gov/priority/conservation/pdfs/FINAL_List_of_165_PCAs_Approved_as_of_September_2015.pdf

In [ ]:
designation['NL'] = designation['Designation'].str.extract('(NL)')
designation['AL'] = designation['Designation'].str.extract('(AL)')
designation['UG'] = designation['Designation'].str.extract('(UG)')
designation['RR'] = designation['Designation'].str.extract('(RR)')

In [ ]:
data = pd.merge(data, designation, left_on='joinkey', right_on='Area ID', how='left')

In [ ]:
data.to_csv('../pca/pca_vis_designation.csv', index=False)

## PCA x SLR scenarios

In [2]:
import glob
from functools import reduce
from bokeh.charts import show, output_file

In [3]:
def join_scenario_puds(scenario, whole_pca_df):
    print(scenario)
    
    # table with userdays in the scenario polygons, keyed by pid
    spud = pd.read_csv(os.path.join(scenario, "userdays_avg_annual_bypid_2005_2015.csv"))
    
    # table linking userdays id (pid) and scenario polygon id (FID_spjoin)
    # sometimes this table includes an 'area' column sometimes not, we won't use it
    pid_fidspjoin_crswlk = pd.read_csv(glob.glob(os.path.join(scenario, '*_pid'))[0])
    pid_fidspjoin_crswlk = pid_fidspjoin_crswlk[['FID_spjoin', 'pid']]
    
    # table linking scenario polygon id (TARGET_FID) and master PCA unique id (joinkey, fipco)
    fidspjoin_joinkey_crswlk = gpd.read_file(glob.glob(os.path.join(scenario, 'spjoin*.dbf'))[0])
    fidspjoin_joinkey_crswlk = fidspjoin_joinkey_crswlk[['TARGET_FID', 'joinkey', 'fipco', 'name']]
    
    # joins to get scenario userdays linked to master PCA unique id (joinkey, fipco)
    spud_fid = pd.merge(spud, pid_fidspjoin_crswlk, on='pid')
    np.all(np.isnan(spud_fid['avg_ann_ud']))
    dat = pd.merge(spud_fid, fidspjoin_joinkey_crswlk, left_on='FID_spjoin', right_on='TARGET_FID')
    
    ## join scenario PUDs to whole PUDs
    dat2 = pd.merge(dat, whole_pca_df, on=['joinkey', 'fipco'], how='right')
    
    scen = os.path.basename(scenario)
    # proportion of PUD that are outside the flood zone
    dat2[scen] = 1 - dat2['avg_ann_ud']/dat2['pud']
    return(dat2[['joinkey', 'fipco', scen]])

In [4]:
data = pd.read_csv('../pca/pca_vis_designation.csv', dtype={'fipco': 'str'})

In [5]:
data['uid'] = data['joinkey']+data['fipco']

In [6]:
basedir = '../bcdc_slr/PCAexposure_201710/slr_scenarios/'
# for now excluding the FEMA scenarios since they have different unique ID column name in diss.shp
scenarios = ['12inches', '24inches', '36inches',  '48inches', '52inches', '66inches', '77inches', '84inches', '96inches', '108inches']

In [27]:
results = []
for scen in scenarios:
    results.append(join_scenario_puds(os.path.join(basedir, scen), data))

# make single table
slrdata = reduce(lambda x, y: pd.merge(x, y, on = ['joinkey', 'fipco']), results)

# single key for PCAs
slrdata['uid'] = slrdata['joinkey']+slrdata['fipco']
slrdata.drop(labels=['joinkey', 'fipco'], axis='columns', inplace=True)
slrdata.set_index('uid', inplace=True)
slrdata['0inches'] = 1
scenario_labels = ['0inches'] + scenarios
slrdata = slrdata[scenario_labels]
slrdata.fillna(1, inplace=True)

numeric_index = [int(s.strip('inches')) for s in list(slrdata)]
numeric_index

# print(slrdata.head())

# # transpose and use SLR number as row key
# tslrdata = pd.DataFrame.transpose(slrdata)
# numeric_index = [int(s.strip('inches')) for s in tslrdata.index.values]
# tslrdata['slr'] = numeric_index
# tslrdata.set_index('slr', inplace=True)

# # set the 0 slr scenario to 100 percent of PUDs
# allones = pd.DataFrame.transpose(pd.DataFrame(data=np.ones_like(tslrdata.iloc[0]), index=list(tslrdata)))
# tslrdata = tslrdata.append(allones)
# tslrdata = tslrdata.reindex([0, 12, 24, 36, 48, 52, 66, 77, 84, 96, 108])

# # # NAs represent no intersection between PCA and SLR polygon, so no change from 100% PUD
# tslrdata.fillna(1, inplace=True)
# # tslrdata

../bcdc_slr/PCAexposure_201710/slr_scenarios/12inches
../bcdc_slr/PCAexposure_201710/slr_scenarios/24inches
../bcdc_slr/PCAexposure_201710/slr_scenarios/36inches
../bcdc_slr/PCAexposure_201710/slr_scenarios/48inches
../bcdc_slr/PCAexposure_201710/slr_scenarios/52inches
../bcdc_slr/PCAexposure_201710/slr_scenarios/66inches
../bcdc_slr/PCAexposure_201710/slr_scenarios/77inches
../bcdc_slr/PCAexposure_201710/slr_scenarios/84inches
../bcdc_slr/PCAexposure_201710/slr_scenarios/96inches
../bcdc_slr/PCAexposure_201710/slr_scenarios/108inches


[0, 12, 24, 36, 48, 52, 66, 77, 84, 96, 108]

In [23]:
[numeric_index]*len(pcaslr.index.values)

[[0, 12, 24, 36, 48, 52, 66, 77, 84, 96, 108],
 [0, 12, 24, 36, 48, 52, 66, 77, 84, 96, 108],
 [0, 12, 24, 36, 48, 52, 66, 77, 84, 96, 108],
 [0, 12, 24, 36, 48, 52, 66, 77, 84, 96, 108],
 [0, 12, 24, 36, 48, 52, 66, 77, 84, 96, 108],
 [0, 12, 24, 36, 48, 52, 66, 77, 84, 96, 108],
 [0, 12, 24, 36, 48, 52, 66, 77, 84, 96, 108],
 [0, 12, 24, 36, 48, 52, 66, 77, 84, 96, 108],
 [0, 12, 24, 36, 48, 52, 66, 77, 84, 96, 108],
 [0, 12, 24, 36, 48, 52, 66, 77, 84, 96, 108],
 [0, 12, 24, 36, 48, 52, 66, 77, 84, 96, 108],
 [0, 12, 24, 36, 48, 52, 66, 77, 84, 96, 108],
 [0, 12, 24, 36, 48, 52, 66, 77, 84, 96, 108],
 [0, 12, 24, 36, 48, 52, 66, 77, 84, 96, 108],
 [0, 12, 24, 36, 48, 52, 66, 77, 84, 96, 108],
 [0, 12, 24, 36, 48, 52, 66, 77, 84, 96, 108],
 [0, 12, 24, 36, 48, 52, 66, 77, 84, 96, 108],
 [0, 12, 24, 36, 48, 52, 66, 77, 84, 96, 108],
 [0, 12, 24, 36, 48, 52, 66, 77, 84, 96, 108],
 [0, 12, 24, 36, 48, 52, 66, 77, 84, 96, 108],
 [0, 12, 24, 36, 48, 52, 66, 77, 84, 96, 108],
 [0, 12, 24, 

In [13]:
slrdata.head()

,0inches,12inches,24inches,36inches,48inches,52inches,66inches,77inches,84inches,96inches,108inches
uid,,,,,,,,,,,
MR46041,1,0.993022,0.988835,0.988137,0.862526,0.860433,0.847872,0.838102,0.833217,0.832519,0.832519
MR156041,1,0.381696,0.357143,0.341518,0.053571,0.037946,0.015625,0.008929,0.006696,0.004464,0.004464
MR186041,1,0.453488,0.348837,0.290698,0.168605,0.162791,0.110465,0.110465,0.110465,0.098837,0.087209
MR146041,1,1.000000,1.000000,1.000000,0.690722,0.680412,0.592784,0.587629,0.587629,0.587629,0.587629
MR96041,1,0.725108,0.461039,0.225108,0.099567,0.075758,0.058442,0.049784,0.043290,0.043290,0.041126


In [16]:
# join some PCA metadata
pcaslr = pd.merge(slrdata, data[['uid', 'pud', 'name', 'Designation']], left_index=True, right_on='uid', how='left')

In [28]:
output_notebook()

# hover = HoverTool(tooltips=[("PCA", "@uid")])
# source = ColumnDataSource(data=dict(
#             xs=[tslrdata.index.values]*numlines,
#             ys=[tslrdata[name].values for name in tslrdata],
#             pcas=list(tslrdata)
#         )) 



source = ColumnDataSource(data=dict(
            xs=[numeric_index]*len(pcaslr.index.values),
            ys=[pcaslr.loc[i, scenario_labels] for i in pcaslr.index.values],
            name=list(pcaslr['name']),
            pud=list(pcaslr['pud']),
            desig=list(pcaslr['Designation'])
        )) 

p = figure(width=900, height=400, x_range=(-5,110)) 
p.multi_line(xs='xs',
                ys='ys',
                source=source,
                line_color='red',
                line_alpha=0.4,
                line_width=1)

# p = figure(width=900, height=400) 
# p.multi_line(xs=[tslrdata.index.values]*numlines,
#                 ys=[tslrdata[name].values for name in tslrdata],
#                 line_color='red',
#                 line_alpha=0.4,
#                 line_width=1)

p.add_tools(HoverTool(show_arrow=False, line_policy='next', tooltips=[
    ('PCA', '@name'),
    ('PUD', '@pud'),
    ('Type', '@desig')
]))

# p = TimeSeries(tslrdata, x='uid', legend=True,
#                title="", ylabel='')

show(p, notebook_handle=True)

Loading BokehJS ...

/home/dmf/miniconda3/envs/geowork/lib/python3.6/site-packages/bokeh/core/json_encoder.py:73: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  if pd and isinstance(obj, pd.tslib.Timestamp):


## Explore PCA SLR inundation -- NOAA 6FT

What percentage of each PCA (and the network as a whole) is vulnerable to SLR inundation?  
And how does recreation importance of PCAs vary with SLR vulnerability?

In [ ]:
# SLR shapefiles, downloaded as .gdb from https://coast.noaa.gov/slrdata/
# explored in QGIS and exported shps for appropriate layers
slr = gpd.read_file('../slr/CA_MTR23_slr_6ft.shp')
slr.crs

In [ ]:
base = pca_shp.plot(color='green', edgecolor=None, figsize=(8,8))
slr.plot(ax=base, color='blue', alpha=0.5)

#### load table with area-inundated calculations - produced by pca_slr_intersect.py

In [ ]:
inundation = pd.read_csv('pca_area_inundated_slr6ft.csv')

In [ ]:
inundation.head()

In [ ]:
inundation.shape[0] == data.shape[0]

In [ ]:
data['flood_area'] = inundation['area_m2']
data['pr_flooded'] = data['flood_area'] / data['area_m2']

### PCAs most inundated -- sorted by proportion of their area that would flood with 6ft of SLR

In [ ]:
data[['name', 'pud', 'tud', 'area_m2', 'flood_area', 'pr_flooded']].sort_values(by='pr_flooded', ascending=False).head(15)

### SLR vulnerability related to Visitation Rates

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(14,5))
data.plot.scatter(x='pr_flooded', y='logpud', ax=axs[0])
data.plot.scatter(x='pr_flooded', y='logtud', ax=axs[1])

In [ ]:
output_notebook()

source = ColumnDataSource(data)
hover1 = HoverTool(tooltips=[("name", "@name")])
hover2 = HoverTool(tooltips=[("name", "@name")])

p = figure(plot_width=400, plot_height=400, tools=[hover1],
           title="SLR vulnerability related to Visitation Rates")

p.circle('pr_flooded', 'pud_per_m2', size=5, color='blue', alpha=0.5, source=source)
p.xaxis.axis_label = 'pr_flooded'
p.yaxis.axis_label = 'pud_per_m2'

p2 = figure(plot_width=400, plot_height=400, tools=[hover2],
           title="SLR vulnerability related to Visitation Rates")

p2.circle('pr_flooded', 'tud_per_m2', size=5, color='red', alpha=0.5, source=source)
p2.xaxis.axis_label = 'pr_flooded'
p2.yaxis.axis_label = 'tud_per_m2'

show(row(p, p2), notebook_handle=True)

### PCAs with greater than 30% area flooded and relatively dense visitation

In [ ]:
(data[['name', 'pud_per_m2', 'area_m2', 'pr_flooded']]
.loc[(data['pr_flooded'] > 0.3) & (data['pudden_tile'] >= 8)]
.sort_values(by='pud_per_m2', ascending=False)
)

In [ ]:
(data[['name', 'tud_per_m2', 'area_m2', 'pr_flooded']]
.loc[(data['pr_flooded'] > 0.3) & (data['tudden_tile'] >= 8)]
.sort_values(by='tud_per_m2', ascending=False)
)

### Export Table

In [ ]:
data.to_csv('../pca/pca_vis_slr_designation.csv', index=False)

In [ ]:
data.head()

#### joining data to shp to make some maps in qgis

In [ ]:
data = pd.read_csv('../pca/pca_vis_slr_designation.csv')
shp = gpd.read_file('../twitter/shp/Priority_Conservation_Areas_current_dissolve_pid.shp')

In [ ]:
shp = shp.merge(data, on='pid')

In [ ]:
shp.to_file('../pca/pca_current_visdata.shp')

### How much of entire PCA network will be flooded?
- there's some overlap in PCAs, for now we're ignoring it.
- What portion of the PUDs in the network are in places that will be flooded? begs for a finer resolution PUD grid.

#### Percent of PCA area inundated with a 6ft SLR scenario

In [ ]:
sum(data['flood_area'])/sum(data['area_m2'])*100